In [17]:
import os
from pathlib import Path

import huggingface_hub
import datasets

### Upload previously corrupted videos (in sub directory 3320-3954)

In [2]:
src_dir = "C:/Users/Jia Herng/Desktop/3320-3954"
path_in_repo = "data/video/3320-3954"
hf_api = huggingface_hub.HfApi()

uploaded_url = hf_api.upload_folder(
    folder_path=src_dir,
    path_in_repo=path_in_repo,
    repo_id="jherng/xd-violence",
    repo_type="dataset",
    commit_message="Upload leftover videos",
)

print(uploaded_url)

Your.Name.2016__#00-43-52_00-46-55_label_A.mp4:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

Your.Name.2016__#00-51-55_00-55-16_label_A.mp4:   0%|          | 0.00/14.3M [00:00<?, ?B/s]

Your.Name.2016__#00-47-00_00-49-30_label_A.mp4:   0%|          | 0.00/14.1M [00:00<?, ?B/s]

Upload 8 LFS files:   0%|          | 0/8 [00:00<?, ?it/s]

Your.Name.2016__#00-36-52_00-41-00_label_A.mp4:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

Your.Name.2016__#00-56-40_01-01-02_label_A.mp4:   0%|          | 0.00/24.9M [00:00<?, ?B/s]

Your.Name.2016__#01-05-30_01-08-59_label_A.mp4:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

Your.Name.2016__#01-09-30_01-16-59_label_A.mp4:   0%|          | 0.00/44.7M [00:00<?, ?B/s]

Your.Name.2016__#01-17-00_01-19-40_label_A.mp4:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

https://huggingface.co/datasets/jherng/xd-violence/tree/main/data/video/3320-3954


### Generate new train_list.txt

In [15]:
repo_path = "datasets/jherng/xd-violence/"
video_path = Path(os.path.join(repo_path, "data/video")).as_posix()
subdirs = ["1-1004", "1005-2004", "2005-2804", "2805-3319", "3320-3954"]
hf_fs = huggingface_hub.HfFileSystem()

with open(os.path.join("train_list.txt"), "w") as f:
    for subdir in subdirs:
        subdir_path = Path(os.path.join(video_path, subdir, "*.mp4")).as_posix()
        subdir_files = hf_fs.glob(subdir_path)
        subdir_files = [fp.split(video_path)[-1].lstrip("/").rstrip(".mp4") for fp in subdir_files]
        for file in subdir_files:
            f.write(f"{file}\n")
        

### Generate a test_list.txt

In [16]:
repo_path = "datasets/jherng/xd-violence/"
video_path = Path(os.path.join(repo_path, "data/video")).as_posix()
subdir = "test_videos"
hf_fs = huggingface_hub.HfFileSystem()

with open(os.path.join("test_list.txt"), "w") as f:
    subdir_path = Path(os.path.join(video_path, subdir, "*.mp4")).as_posix()
    subdir_files = hf_fs.glob(subdir_path)
    subdir_files = [fp.split(video_path)[-1].lstrip("/").rstrip(".mp4") for fp in subdir_files]
    for file in subdir_files:
        f.write(f"{file}\n")
        

### Fix dataset loading script

In [38]:
ds = datasets.load_dataset("jherng/xd-violence", "video", split="test", streaming=True)
for i, x in enumerate(ds):
    if i == 10:
        break
    print(i, x)
    

0 {'id': 'A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A', 'path': 'https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/test_videos/A.Beautiful.Mind.2001__%2300-25-20_00-29-20_label_A.mp4', 'binary_target': 0, 'multilabel_target': [0], 'frame_annotations': {'start': [], 'end': []}}
1 {'id': 'A.Beautiful.Mind.2001__#00-40-52_00-42-01_label_A', 'path': 'https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/test_videos/A.Beautiful.Mind.2001__%2300-40-52_00-42-01_label_A.mp4', 'binary_target': 0, 'multilabel_target': [0], 'frame_annotations': {'start': [], 'end': []}}
2 {'id': 'A.Beautiful.Mind.2001__#00-50-26_00-55-16_label_A', 'path': 'https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/test_videos/A.Beautiful.Mind.2001__%2300-50-26_00-55-16_label_A.mp4', 'binary_target': 0, 'multilabel_target': [0], 'frame_annotations': {'start': [], 'end': []}}
3 {'id': 'A.Beautiful.Mind.2001__#01-14-30_01-16-59_label_A', 'path': 'ht

### Check progress of feature extraction

In [39]:
repo_path = "datasets/jherng/xd-violence/"
feature_path = Path(os.path.join(repo_path, "data/i3d_rgb")).as_posix()
subdirs = ["1-1004", "1005-2004", "2005-2804", "2805-3319", "3320-3954", "test_videos"]
progress = {}

hf_fs = huggingface_hub.HfFileSystem()

for subdir in subdirs:
    subdir_path = Path(os.path.join(feature_path, subdir, "*.npy")).as_posix()
    subdir_files = hf_fs.glob(subdir_path)
    progress[subdir] = len(subdir_files)

print(progress)

{'1-1004': 1004, '1005-2004': 979, '2005-2804': 737, '2805-3319': 455, '3320-3954': 591, 'test_videos': 493}


In [ ]:
# Current dataset on hf
# test: 800
# train: 3950
# - 1-1004: 1004
# - 1005-2004: 996 (should be 1000 but 4 corrupted)
# - 2005-2804: 800
# - 2805-3319: 515
# - 3320-3954: 635
# total: 4750 (should be 4754 but 4 corrupted)

# Original dataset
# test: 800 (500 violent, 300 non-violent)
# train: 3954
# total: 4754

In [ ]:
# Feature extraction progress
# test: 493/800
# train: 3766/3950
# - 1-1004: 1004/1004
# - 1005-2004: 979/996
# - 2005-2804: 737/800
# - 2805-3319: 455/515
# - 3320-3954: 591/635
# all: 4259/4750